<h1>Содержание<span class="tocSkip"></span></h1>
<br>
<div class="toc">
    <ul class="toc-item">
        <li>
            <span>
                <a href="#1-Подготовка-окружения">
                    <span class="toc-item-num">1&nbsp;&nbsp;</span>
                    Подготовка окружения
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#2-Загрузка-данных">
                    <span class="toc-item-num">2&nbsp;&nbsp;</span>
                    Загрузка данных
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#3-Трансформерная-архитектура-GPT">
                    <span class="toc-item-num">3&nbsp;&nbsp;</span>
                    Трансформерная архитектура GPT
                </a>
            </span>
        </li>
        <li>
            <span>
                <a href="#4-Общий-вывод">
                    <span class="toc-item-num">4&nbsp;&nbsp;</span>
                    Общий вывод
                </a>
            </span>
        </li>
    </ul>
</div>

# Генеративные текстовые нейросети | Архитектура GPT

**Постановка задачи:** натренировать и сравнить качество нескольких генеративных текстовых моделей на одном из заданных текстовых датасетов.

**Источник данных:** [Harry Potter and the Methods of Rationality](https://hpmor.ru/).

**Характер данных:** текст книги "Гарри Поттер и методы рационального мышления".

**Основные этапы:** исследовать следующие нейросетевые архитектуры:

1. Simple RNN с посимвольной и пословной токенизацией.
2. Однонаправленная однослойная и многослойная LSTM c посимвольной токенизацией и токенизацией по словам и [на основе BPE](https://keras.io/api/keras_nlp/tokenizers/byte_pair_tokenizer/).
3. Двунаправленная LSTM.
4. *(На хорошую оценку)* трансформерная архитектура (GPT) "с нуля" [пример](https://keras.io/examples/generative/text_generation_gpt/).
5. *(На отличную оценку)* дообучение предобученной GPT-сети [пример](https://github.com/ZotovaElena/RuGPT3_finetuning).

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

# Реализации

1. [RNN с посимвольной токенизацией](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/RNN_char.ipynb)
2. [RNN с пословной токенизацией](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/RNN_word.ipynb)
3. [Однонаправленная LSTM + BPE](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/LSTM_unidirectional_BPE.ipynb)
4. [Двунаправленная LSTM](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/LSTM_bidirectional.ipynb)
5. Архитектура GPT (текущий файл)
6. [Дообучение GPT](https://github.com/MAILabs-Edu-2023/magai_lab3_gennn-nlp_lab/blob/main/GPT_finetuning.ipynb)

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 1 Подготовка окружения

Импорт библиотек:

In [1]:
from typing import Tuple, List

import os

import tensorflow as tf

from tf.data import TextLineDataset, AUTOTUNE
from tf.keras.losses import SparseCategoricalCrossentropy

from tensorflow import keras

import keras_nlp

from keras_nlp.tokenizers import WordPieceTokenizer, compute_word_piece_vocabulary
from keras_nlp.layers import StartEndPacker, TokenAndPositionEmbedding, TransformerDecoder
from keras_nlp.metrics import Perplexity
from keras_nlp.samplers import GreedySampler, BeamSampler, RandomSampler, TopKSampler, TopPSampler

from utils.useful_funcs import request_url, get_url_data, get_data

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 2 Загрузка данных

Задание функции, разделяющей набор данных на выборки:

In [2]:
def train_test_split(text: list, test_size: float) -> Tuple[List[str], List[str]]:
    random.shuffle(text)
    threshold = int((1 - test_size) * len(text))
    
    train = text[:threshold]
    test = text[threshold:]
    
    return train, test

Задание функции, сохраняющей разделённые выборки в файлы:

In [3]:
def split_into_train_valid_test(text: str,
                                path_train: str,
                                path_valid: str,
                                path_test: str = None,
                                test_size: float = 0.25) -> None:
    
    if os.path.isfile(path_train) == False and os.path.isfile(path_valid) == False:
        
        text_split = [s.strip() for s in text.split('.')]
        
        train_text, valid_text = train_test_split(text_split, test_size)
        test_text = None
        
        if path_test != None:
            valid_text, test_text = train_test_split(valid_text, test_size)
            test_text = ' '.join(test_text)
        
        train_valid_test = [(path_train, train_text), (path_valid, valid_text), (path_test, test_text)]
        
        for (path, text) in train_valid_test:
            if path != None:
                with open(path, 'w', encoding='utf-8') as file:
                    file.write(' '.join(text))
        
        print('Splitted into:', len(train_text), 'train,', len(valid_text), 'valid and', 
              len(test_text) if test_text != None else 0, 'test')
    
    else:
        print('Files already exist')

---

Проверка наличия папки для хранения наборов данных:

In [4]:
if os.path.isdir('data/') == False:
    os.mkdir('data/')

Задание пути к файлу с основным набором данных:

In [5]:
path_file = 'data/hpmor.txt'

Формирование/загрузка набора данных в зависимости от его наличия:

In [6]:
try:
    with open(path_file, 'r', encoding='utf-8') as file:
        text = file.read()
    
    print('Uploaded from', path_file)
    
except:
    text = get_data('https://hpmor.ru/')
    
    with open(path_file, 'w', encoding='utf-8') as file:
        file.write(text)
    
    print('Saved to', path_file)

Uploaded from data/hpmor.txt


Выведение на экран начала текста:

In [7]:
text[:500]

'гарри поттер и методы рационального мышления. элиезер юдковский (less wrong). петуния вышла замуж не за дурсля, а за университетского профессора, и гарри попал в гораздо более благоприятную среду. у него были частные учителя, дискуссии с отцом, а главное — книги, сотни и тысячи научных и фантастических книг. в 11 лет гарри знаком с квантовой механикой, когнитивной психологией, теорией вероятностей и другими вещами. но гарри не просто вундеркинд, у него есть загадочная тёмная сторона, которая явн'

Выведение на экран общего числа слов в тексте:

In [8]:
print('Всего слов:', len(text.split(' ')))

Всего слов: 559855


Задание путей до тренировочной и валидационной выборок:

In [9]:
path_train = 'data/hpmor_train.txt'
path_valid = 'data/hpmor_valid.txt'

Разделение текста на тренировочную и валидационную выборки:

In [10]:
split_into_train_valid_test(text, path_train, path_valid, test_size=0.25)

Splitted into: 28013 train, 9338 valid and 0 test


<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 3 Трансформерная архитектура GPT

Задание функции, разделяющей входящее значение на признаки и их

In [11]:
def preprocess(inputs: tf.Tansor) -> Tuple[tf.Tensor, tf.Tensor]:
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    
    return features, labels

Задание функции, выводящей минимальную и максимальную длины предложений во всём тексте:

In [ ]:
def print_max_min_string(text: str) -> None:
    print('Максимальная длина строки:', len(max(text.split('.'), key=len).split()))
    print('Минимальная длина строки:', len(min(text.split('.'), key=len).split()))

---

Выведение на экран максимальной и минимальной длин предложений:

In [12]:
print_max_min_string(text)

Максимальная длина строки: 217
Минимальная длина строки: 0


In [13]:
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000

In [14]:
train_dataset = (
    TextLineDataset(path_train)
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

valid_dataset = (
    TextLineDataset(path_valid)
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)

In [15]:
list(train_dataset.as_numpy_iterator())[0][0].decode('utf-8')[:500]

'— но что-то пошло не так, — сказал гарри как только мистер квиррелл вошёл в комнату, маленький оранжевый огонёк померк и начал мигать, как свеча на ветру когда гарри выяснил ответ, он очень удивился на кровати снова возник человек вообще-то даже целых две и картина парка, через который, как нам кажется, мы идём, появляется внутри нашего мозга после обработки сигналов от сетчатки глаз фениксы лишены мудрости, гарри, у них нет возможности судить о правильности нашего выбора за день до этого гарри '

In [16]:
vocabulary = compute_word_piece_vocabulary(
    train_dataset,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=['[PAD]', '[UNK]', '[BOS]'],
)

In [17]:
print('Слов в словаре:', len(vocabulary))

Слов в словаре: 4919


In [18]:
vocabulary[:10]

['[PAD]', '[UNK]', '[BOS]', '!', '#', '$', '%', '&', '(', ')']

In [19]:
tokenizer = WordPieceTokenizer(
    vocabulary=vocabulary,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

In [20]:
start_packer = StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id('[BOS]'),
)

In [21]:
train = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
valid = valid_dataset.map(preprocess, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

In [22]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)

In [23]:
embedding_layer = TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)

In [24]:
x = embedding_layer(inputs)

In [25]:
for _ in range(NUM_LAYERS):
    decoder_layer = TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)

In [26]:
outputs = keras.layers.Dense(VOCAB_SIZE)(x)

In [27]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [28]:
LOSS_F = SparseCategoricalCrossentropy(from_logits=True)
PERPLEX = Perplexity(from_logits=True, mask_token_id=0)

In [29]:
model.compile(optimizer='adam', loss=LOSS_F, metrics=[PERPLEX])

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

In [31]:
N_EPOCHS = 100

In [32]:
model.fit(train, validation_data=valid, verbose=2, epochs=N_EPOCHS)

Epoch 1/100
1/1 - 6s - loss: 8.5784 - perplexity: 5315.5464 - val_loss: 8.4019 - val_perplexity: 4455.5552 - 6s/epoch - 6s/step
Epoch 2/100
1/1 - 2s - loss: 8.2647 - perplexity: 3884.2444 - val_loss: 8.2050 - val_perplexity: 3659.1411 - 2s/epoch - 2s/step
Epoch 3/100
1/1 - 2s - loss: 7.9541 - perplexity: 2847.1301 - val_loss: 7.9828 - val_perplexity: 2930.1980 - 2s/epoch - 2s/step
Epoch 4/100
1/1 - 2s - loss: 7.6482 - perplexity: 2096.8628 - val_loss: 7.7708 - val_perplexity: 2370.3252 - 2s/epoch - 2s/step
Epoch 5/100
1/1 - 2s - loss: 7.3716 - perplexity: 1590.2134 - val_loss: 7.5927 - val_perplexity: 1983.7341 - 2s/epoch - 2s/step
Epoch 6/100
1/1 - 2s - loss: 7.1279 - perplexity: 1246.2792 - val_loss: 7.4517 - val_perplexity: 1722.8259 - 2s/epoch - 2s/step
Epoch 7/100
1/1 - 2s - loss: 6.9130 - perplexity: 1005.3046 - val_loss: 7.3411 - val_perplexity: 1542.4810 - 2s/epoch - 2s/step
Epoch 8/100
1/1 - 2s - loss: 6.7168 - perplexity: 826.1465 - val_loss: 7.2562 - val_perplexity: 1416.796

Epoch 66/100
1/1 - 2s - loss: 0.3890 - perplexity: 1.4756 - val_loss: 7.8745 - val_perplexity: 2629.4885 - 2s/epoch - 2s/step
Epoch 67/100
1/1 - 2s - loss: 0.3667 - perplexity: 1.4430 - val_loss: 7.8983 - val_perplexity: 2692.6409 - 2s/epoch - 2s/step
Epoch 68/100
1/1 - 2s - loss: 0.3431 - perplexity: 1.4093 - val_loss: 7.9457 - val_perplexity: 2823.3010 - 2s/epoch - 2s/step
Epoch 69/100
1/1 - 2s - loss: 0.3237 - perplexity: 1.3822 - val_loss: 7.9804 - val_perplexity: 2923.1338 - 2s/epoch - 2s/step
Epoch 70/100
1/1 - 2s - loss: 0.3055 - perplexity: 1.3572 - val_loss: 8.0089 - val_perplexity: 3007.6089 - 2s/epoch - 2s/step
Epoch 71/100
1/1 - 2s - loss: 0.2858 - perplexity: 1.3308 - val_loss: 8.0375 - val_perplexity: 3094.8586 - 2s/epoch - 2s/step
Epoch 72/100
1/1 - 2s - loss: 0.2718 - perplexity: 1.3123 - val_loss: 8.0537 - val_perplexity: 3145.5359 - 2s/epoch - 2s/step
Epoch 73/100
1/1 - 2s - loss: 0.2551 - perplexity: 1.2906 - val_loss: 8.0925 - val_perplexity: 3269.7981 - 2s/epoch - 

In [33]:
prompt_tokens = start_packer(tokenizer(['']))

In [34]:
prompt_tokens

<tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [35]:
def next(prompt: tf.Tensor, 
         cache: tf.Tensor, 
         index: tf.Tensor) -> Tuple[tf.Tensor, None, Tuple]:
    
    logits = model(prompt)[:, index - 1, :]
    hidden_states = None
    
    return logits, hidden_states, cache

In [36]:
def train_model(sampler: keras_nlp.samplers,
                name: str,
                prompt_tokens: tf.Tensor = prompt_tokens) -> str:
    
    tokens = sampler(
        next=next,
        prompt=prompt_tokens,
        index=1,
    )
    
    text = tokenizer.detokenize(tokens)
    
    print(f'{name.capitalize()} search generated text:')
    return text.numpy()[0].decode('utf-8')

---

Greedy search

In [37]:
train_model(GreedySampler(), 'Greedy')

Greedy search generated text:


'[BOS] ищу , как всем его избегнуть ! и рая тоже , ибо я подозреваю , тот рай , что создал сумасшедший бог , лишь жалок по сравненью с небесами , что будут на земле , как только здесь бессмертным королём я стану мальчик смотрел на неё какой - то врождённый талант ? нельзя научиться быть метаморфомагом . . . и всё же это не похоже на силу , что неведома ему и хранил его в тайне на случай , если вдруг придётся проверять свою'

---

Beam search

In [38]:
train_model(BeamSampler(num_beams=10), 'Beam')

Beam search generated text:


'[BOS] книга « думай как физик » в оригинале называется « thinking physics » чуть - чуть — у меня тут теперь будет синяк — этим утром я вспомню это в следующий раз , когда вознамерюсь взять на себя вину за что - нибудь из - за угла выскочила низенькая коренастая фигура профессора травоведения просто на случай , если магия гарри полностью истощится при встрече с самым тёмным из всех созданий он сел на'

---

Random search

In [39]:
train_model(RandomSampler(), 'Random')

Random search generated text:


'[BOS] осталось ходов : 2 — минус три миллиона тёмным изц залеками , маленький оране защиты ,надцатилетний мальчик , в смысле . . . — не надо , наблюдения , что еслипорить , делать взрывчатку в домашних условиях гермиона устала держать голову так , нет , вне камереевирным могла просто присесть рядом и тоже открыть книгу а что ему оставалось ? сказать « нет » ? — хорошо план забини заключался в том , чтобы на навернякапилаеписку от младшего брата , второй защитит вашу'

---

Top-K search

In [40]:
train_model(TopKSampler(k=10), 'Top-K')

Top-k search generated text:


'[BOS] ищу , как всем его избегнуть ! и рая тоже , ибо я подозреваю , тот рай с -нит , живший в xics » чуть - чуть — у меня тут теперь будет с небесами , может быть , откуда открывался прекрамертным королём я стану мальчик смотрел на неё какой - то врождённый талант ? нельзя научиться быть мета — но , что след , как идёт не сицей и посоветчатки глаз фениксы лишены мудрости , бесконечное и пустое ничто с бан'

---

Top-P search

In [41]:
train_model(TopPSampler(p=0.5), 'Top-P')

Top-p search generated text:


'[BOS] осталось ходов : 2 — минус три миллиона баллов ? — возмутился гарри и если это было лучшей стратегией спасения жизни своего ребёнка , которую она могла придумать , то произошедшее — её окончательный провал как матери гарри даже собирался предложить ей вторую по значимости должность в группе борцов против тёмного лорда , но был не настолько глуп , чтобы озвучить эту мысль вслух это всё , профессор ? снова возникла пауза закончится ли текст на листке верным или ложным заключением опре'

<div style="background-color: blue; height: 2px; margin: 10px 0;"></div>

## 4 Общий вывод

<div style="text-align: center; font-size: 20px; padding: 15px 0;">
    <a href="#Содержание" data-toc-modified-id="Содержание" style="text-decoration: none; color: #296eaa; border: 2px dashed #296eaa; opacity: 0.8; border-radius: 3px; padding: 10px 80px;">
        В начало файла ↑
    </a>
</div>